## Importing the relevant packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.arima_model import ARIMA
from scipy.stats.distributions import chi2 
from arch import arch_model
from math import sqrt
import seaborn as sns
sns.set()

## Importing the Data and Pre-processing 

In [3]:
raw_csv_data = pd.read_csv("inputs\Index2018.csv") 
df_comp=raw_csv_data.copy()
df_comp.date = pd.to_datetime(df_comp.date, dayfirst = True)
df_comp.set_index("date", inplace=True)
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

In [4]:
df_comp['market_value']=df_comp.ftse

In [5]:
del df_comp['spx']
del df_comp['dax']
del df_comp['ftse']
del df_comp['nikkei']
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

In [6]:
import warnings
warnings.filterwarnings("ignore")

## The LLR Test

In [7]:
def LLR_test(mod_1, mod_2, DF = 1):
    L1 = mod_1.fit(start_ar_lags = 11).llf
    L2 = mod_2.fit(start_ar_lags = 11).llf
    LR = (2*(L2-L1))    
    p = chi2.sf(LR, DF).round(3)
    return p

## Creating Returns

In [8]:
df['returns'] = df.market_value.pct_change(1)*100

## The Simple GARCH Model

In [9]:
model_garch_1_1 = arch_model(df.returns[1:],mean="Constant", vol='GARCH', p=1, q=1)
results_garch_1_1 = model_garch_1_1.fit()
results_garch_1_1.summary()

Iteration:      1,   Func. Count:      6,   Neg. LLF: 6579303469.390623
Iteration:      2,   Func. Count:     15,   Neg. LLF: 2701100877.2298183
Iteration:      3,   Func. Count:     23,   Neg. LLF: 7009.030632045198
Iteration:      4,   Func. Count:     29,   Neg. LLF: 7024.035884053223
Iteration:      5,   Func. Count:     35,   Neg. LLF: 7010.712866814991
Iteration:      6,   Func. Count:     41,   Neg. LLF: 6975.418107495356
Iteration:      7,   Func. Count:     47,   Neg. LLF: 7092.271289251072
Iteration:      8,   Func. Count:     53,   Neg. LLF: 6973.8792679196495
Iteration:      9,   Func. Count:     59,   Neg. LLF: 6970.088049064454
Iteration:     10,   Func. Count:     64,   Neg. LLF: 6970.058478417896
Iteration:     11,   Func. Count:     69,   Neg. LLF: 6970.058367475591
Iteration:     12,   Func. Count:     74,   Neg. LLF: 6970.058366189888
Iteration:     13,   Func. Count:     78,   Neg. LLF: 6970.058366189172
Optimization terminated successfully    (Exit mode 0)
        

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -6970.06
Distribution:                  Normal   AIC:                           13948.1
Method:            Maximum Likelihood   BIC:                           13974.2
                                        No. Observations:                 5020
Date:                Mon, Aug 01 2022   Df Residuals:                     5019
Time:                        17:31:27   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0466  1.183e-02      3.939  8.187e-05 [2.342e-02,6.981e-02]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0109  3.004e-03      3.640  2.724e-04 [5.048e-03,1.682e-02]
alpha[1]       0.0835  1.071e-02      7.794  6.476e-15   [6.249e-02,  0.104]
beta[1]        0.9089  1.148e-02     79.168      0.000     [  0.886,  0.931]
============================================================================

Covariance estimator: robust
"""

## Higher-Lag GARCH Models

In [10]:
model_garch_1_2 = arch_model(df.returns[1:],mean="Constant", vol='GARCH', p=1, q=2)
results_garch_1_2 = model_garch_1_2.fit()
results_garch_1_2.summary()

Iteration:      1,   Func. Count:      7,   Neg. LLF: 136466776694655.33
Iteration:      2,   Func. Count:     17,   Neg. LLF: 808459246.0256559
Iteration:      3,   Func. Count:     25,   Neg. LLF: 10137.459118305613
Iteration:      4,   Func. Count:     33,   Neg. LLF: 7008.430986114414
Iteration:      5,   Func. Count:     40,   Neg. LLF: 6974.173831538361
Iteration:      6,   Func. Count:     46,   Neg. LLF: 6971.511859365692
Iteration:      7,   Func. Count:     52,   Neg. LLF: 6970.616224326323
Iteration:      8,   Func. Count:     58,   Neg. LLF: 6973.892231638968
Iteration:      9,   Func. Count:     65,   Neg. LLF: 6970.063506713137
Iteration:     10,   Func. Count:     71,   Neg. LLF: 6970.058391826686
Iteration:     11,   Func. Count:     77,   Neg. LLF: 6970.058366757141
Iteration:     12,   Func. Count:     83,   Neg. LLF: 6970.05836622724
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6970.05836622724
            Iterations: 12
 

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -6970.06
Distribution:                  Normal   AIC:                           13950.1
Method:            Maximum Likelihood   BIC:                           13982.7
                                        No. Observations:                 5020
Date:                Mon, Aug 01 2022   Df Residuals:                     5019
Time:                        17:35:32   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0466  1.184e-02      3.938  8.219e-05 [2.341e-02,6.982e-02]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0109  2.908e-03      3.761  1.696e-04 [5.236e-03,1.663e-02]
alpha[1]       0.0835  1.189e-02      7.019  2.231e-12   [6.017e-02,  0.107]
beta[1]        0.9089      0.188      4.845  1.268e-06     [  0.541,  1.277]
beta[2]        0.0000      0.180      0.000      1.000     [ -0.352,  0.352]
============================================================================

Covariance estimator: robust
"""

In [11]:
model_garch_2_2 = arch_model(df.returns[1:],mean="Constant", vol='GARCH', p=2, q=2)
results_garch_2_2 = model_garch_2_2.fit()
results_garch_2_2.summary()

Iteration:      1,   Func. Count:      8,   Neg. LLF: 64329.00742578435
Iteration:      2,   Func. Count:     20,   Neg. LLF: 28544.55132855609
Iteration:      3,   Func. Count:     31,   Neg. LLF: 1585503827.2916908
Iteration:      4,   Func. Count:     39,   Neg. LLF: 7021.088669289045
Iteration:      5,   Func. Count:     47,   Neg. LLF: 16912.109253612274
Iteration:      6,   Func. Count:     55,   Neg. LLF: 7130.552772308756
Iteration:      7,   Func. Count:     63,   Neg. LLF: 7023.81939747395
Iteration:      8,   Func. Count:     71,   Neg. LLF: 6986.380333589299
Iteration:      9,   Func. Count:     79,   Neg. LLF: 6971.9124544884
Iteration:     10,   Func. Count:     87,   Neg. LLF: 7863.841245879325
Iteration:     11,   Func. Count:     96,   Neg. LLF: 6967.9766535249855
Iteration:     12,   Func. Count:    104,   Neg. LLF: 6972.8974470021185
Iteration:     13,   Func. Count:    112,   Neg. LLF: 6967.501325334316
Iteration:     14,   Func. Count:    119,   Neg. LLF: 6967.5012

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -6967.50
Distribution:                  Normal   AIC:                           13947.0
Method:            Maximum Likelihood   BIC:                           13986.1
                                        No. Observations:                 5020
Date:                Mon, Aug 01 2022   Df Residuals:                     5019
Time:                        17:35:36   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0465  1.186e-02      3.921  8.824e-05 [2.325e-02,6.973e-02]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0150  4.606e-03      3.246  1.172e-03 [5.922e-03,2.398e-02]
alpha[1]       0.0539  1.635e-02      3.294  9.888e-04 [2.181e-02,8.590e-02]
alpha[2]       0.0550  2.467e-02      2.229  2.578e-02   [6.649e-03,  0.103]
beta[1]        0.7166      0.186      3.845  1.206e-04     [  0.351,  1.082]
beta[2]        0.1641      0.173      0.948      0.343     [ -0.175,  0.503]
============================================================================

Covariance estimator: robust
"""